# Sentiment Classification with a Deep Learning Model

This notebook introduces a machine learning task from the field of natural language processing (machine learning focused on the processing of spoken and written text).

## Sentiment Analysis

The modelled task is a classification task called sentiment analysis. 
Text snippets are classified according to their positive or negative sentiment that is expressed in them. 
This can be modelled as 3-class problem (negative, neutral, positive), or as a degree of sentiment on a 5-class or 10-class scale. 




## Acknowledgement

The notebook is based on https://www.manning.com/books/real-world-natural-language-processing, an upcoming book focused on NLP.

The ML frameworks used are:

* pytorch
* allennlp
* spacy


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/erikgraf/deepLearning/blob/master/Deep_Learning_Sentiment_classifier.ipynb)

## Installing Dependencies

The cell below installs the main dependencies and clones some a repository that forms the basis of the implementation. 

Executing it with `CTRL + Enter` (`STRG +Enter` on a german keyboard) could take a couple of minutes.

In [8]:
!pip install allennlp


     |████████████████████████████████| 7.6MB 4.6MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 204kB 58.5MB/s 
     |████████████████████████████████| 133kB 62.4MB/s 
     |████████████████████████████████| 163kB 63.6MB/s 
     |████████████████████████████████| 245kB 75.2MB/s 
     |████████████████████████████████| 256kB 78.1MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 1.0MB 60.9MB/s 
  Created wheel for ftfy: filename=ftfy-5.6-cp36-none-any.whl size=44553 sha256=b57362a34fbddd3ab1334c75749a6384fc62a0611b9b312b9353934cc92ca757
  Stored in directory: /root/.cache/pip/wheels/43/34/ce/cbb38d71543c408de56f3c5e26ce8ba495a0fa5a28eaaf1046
  Created wheel for numpydoc: filename=numpydoc-0.9.2-cp36-none-any.whl size=31894 sha256=f6cbcaf43dfb46fedf1c678a5a8709ae911c84dced8082951bb4c26cf9cf02d8
  Stored in directory: /root/.cache/pip/wheels/96/f3/52/25c8e1f40637661d27feebc61dae16b84c7cd

In [9]:
!git clone https://github.com/mhagiwara/realworldnlp.git
%cd realworldnlp

Cloning into 'realworldnlp'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 468 (delta 111), reused 107 (delta 50), pack-reused 288
Receiving objects: 100% (468/468), 4.70 MiB | 10.85 MiB/s, done.
Resolving deltas: 100% (240/240), done.
/content/realworldnlp


## Imports

Execute the cell below to load all required modules. 

In [0]:
from typing import Dict

import numpy as np
import torch
import torch.optim as optim
from allennlp.data.dataset_readers.stanford_sentiment_tree_bank import \
    StanfordSentimentTreeBankDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.nn.util import get_text_field_mask
from allennlp.training.metrics import CategoricalAccuracy, F1Measure
from allennlp.training.trainer import Trainer

from realworldnlp.predictors import SentenceClassifierPredictor

## Hyperparameters

The cell below sets the hyperparameters.

* EMBEDDING_DIM: This is the dimensionality of the word embeddings (numeric representations of words such as word2vec or glove (https://nlp.stanford.edu/projects/glove/))
* HIDDEN_DIM: This is the dimensionality of the LSTM (Long Short Term Memory) Deep Learning network. 

A value of 128 is pretty standard for the embeddings and hidden_dim.




In [0]:
EMBEDDING_DIM = 128
HIDDEN_DIM = 128

## Training Data Set

For training we will use the Stanford Sentiment Treebank data set.
A data set for training sentiment analysis models. It is annotated both on the sentence and the word level with regard to the sentiment. 

When loading the data set we can configure the granularity to `'5-class'` or `'3-class'`.

`'3-class'` represents classification on the level of `negative`, `neutral`, `positive` encoded as `0`, `1`, `2` (positive). `'5-class'` on a level from `0` to `4`.





In [0]:
reader = StanfordSentimentTreeBankDatasetReader(granularity='5-class')

In [13]:
train_dataset = reader.read('https://s3.amazonaws.com/realworldnlpbook/data/stanfordSentimentTreebank/trees/train.txt')
dev_dataset = reader.read('https://s3.amazonaws.com/realworldnlpbook/data/stanfordSentimentTreebank/trees/dev.txt')

0it [00:00, ?it/s]
 37%|███▋      | 800768/2160058 [00:00<00:02, 536588.02B/s]
8544it [00:02, 3435.87it/s]
0it [00:00, ?it/s]
 87%|████████▋ | 243712/280825 [00:00<00:00, 531393.08B/s]
1101it [00:01, 1077.33it/s]


## Model Implementation in AllenNLP

Execute the cell below to load the model classification.

Depending on the class level chosen (3 vs 5) change the positive label in the init method to ('2' or '4').


In [0]:
# Model in AllenNLP represents a model that is trained.
@Model.register("lstm_classifier")
class LstmClassifier(Model):
    def __init__(self,
                 word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 vocab: Vocabulary,
                 positive_label: str = '4') -> None:
        super().__init__(vocab)
        # We need the embeddings to convert word IDs to their vector representations
        self.word_embeddings = word_embeddings

        self.encoder = encoder

        # After converting a sequence of vectors to a single vector, we feed it into
        # a fully-connected linear layer to reduce the dimension to the total number of labels.
        self.linear = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))

        # Monitor the metrics - we use accuracy, as well as prec, rec, f1 for 4 (very positive)
        positive_index = vocab.get_token_index(positive_label, namespace='labels')
        self.accuracy = CategoricalAccuracy()
        self.f1_measure = F1Measure(positive_index)

        # We use the cross entropy loss because this is a classification task.
        # Note that PyTorch's CrossEntropyLoss combines softmax and log likelihood loss,
        # which makes it unnecessary to add a separate softmax layer.
        self.loss_function = torch.nn.CrossEntropyLoss()

    # Instances are fed to forward after batching.
    # Fields are passed through arguments with the same name.
    def forward(self,
                tokens: Dict[str, torch.Tensor],
                label: torch.Tensor = None) -> torch.Tensor:
        # In deep NLP, when sequences of tensors in different lengths are batched together,
        # shorter sequences get padded with zeros to make them equal length.
        # Masking is the process to ignore extra zeros added by padding
        mask = get_text_field_mask(tokens)

        # Forward pass
        embeddings = self.word_embeddings(tokens)
        encoder_out = self.encoder(embeddings, mask)
        logits = self.linear(encoder_out)

        # In AllenNLP, the output of forward() is a dictionary.
        # Your output dictionary must contain a "loss" key for your model to be trained.
        output = {"logits": logits}
        if label is not None:
            self.accuracy(logits, label)
            self.f1_measure(logits, label)
            output["loss"] = self.loss_function(logits, label)

        return output

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        precision, recall, f1_measure = self.f1_measure.get_metric(reset)
        return {'accuracy': self.accuracy.get_metric(reset),
                'precision': precision,
                'recall': recall,
                'f1_measure': f1_measure}

## Transform Text into Numeric Representation

The following cells are responsible for the transformation of text in string form into numeric representations that are suitable as learning input for the neural network.

1. Extract vocabulary of unique terms from the text
2. Create embeddings for the terms
3. Define transformation (encoding) for a sequence of text (i.e. a sentence)

In [15]:
# You can optionally specify the minimum count of tokens/labels.
# `min_count={'tokens':3}` here means that any tokens that appear less than three times
# will be ignored and not included in the vocabulary.
vocab = Vocabulary.from_instances(train_dataset + dev_dataset,
                                  min_count={'tokens': 3})

100%|██████████| 9645/9645 [00:00<00:00, 75283.82it/s]


In [0]:
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

In [0]:
# BasicTextFieldEmbedder takes a dict - we need an embedding just for tokens,
# not for labels, which are used as-is as the "answer" of the sentence classification
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

In [0]:
# Seq2VecEncoder is a neural network abstraction that takes a sequence of something
# (usually a sequence of embedded word vectors), processes it, and returns a single
# vector. Oftentimes this is an RNN-based architecture (e.g., LSTM or GRU), but
# AllenNLP also supports CNNs and other simple architectures (for example,
# just averaging over the input vectors).
encoder = PytorchSeq2VecWrapper(
    torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

## Configure Model for Training

The following four cells configure the model for training.

1. The LstmClassifier class takes the word_embeddings, the define sequence encoder and the vocabulary as input configuration. 

2. The BucketIterator is a helper class for iterating over the full training set and randomly selects batches of instances for the training. 

3. optimizer specifies the learning rate for Adam (a mathmatical optimisation function that will guide the weight adaptations of our model).

4. trainer holds our instatiation of the model, and defines the number of epochs.



In [0]:
model = LstmClassifier(word_embeddings, encoder, vocab)

In [0]:
iterator = BucketIterator(batch_size=32, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)

In [0]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [0]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=dev_dataset,
                  patience=40,
                  num_epochs=40)


## Train

Execute the cell below to train.

In [23]:
trainer.train()

accuracy: 0.2611, precision: 0.0000, recall: 0.0000, f1_measure: 0.0000, loss: 1.5802 ||: 100%|██████████| 267/267 [00:10<00:00, 25.49it/s]
accuracy: 0.2579, precision: 0.0000, recall: 0.0000, f1_measure: 0.0000, loss: 1.5735 ||: 100%|██████████| 35/35 [00:00<00:00, 132.12it/s]
accuracy: 0.2805, precision: 0.0000, recall: 0.0000, f1_measure: 0.0000, loss: 1.5652 ||: 100%|██████████| 267/267 [00:10<00:00, 26.36it/s]
accuracy: 0.2534, precision: 0.0000, recall: 0.0000, f1_measure: 0.0000, loss: 1.5744 ||: 100%|██████████| 35/35 [00:00<00:00, 143.35it/s]
accuracy: 0.2773, precision: 0.0000, recall: 0.0000, f1_measure: 0.0000, loss: 1.5583 ||: 100%|██████████| 267/267 [00:09<00:00, 27.27it/s]
accuracy: 0.2616, precision: 0.0000, recall: 0.0000, f1_measure: 0.0000, loss: 1.5659 ||: 100%|██████████| 35/35 [00:00<00:00, 145.85it/s]
accuracy: 0.3017, precision: 0.0000, recall: 0.0000, f1_measure: 0.0000, loss: 1.5282 ||: 100%|██████████| 267/267 [00:09<00:00, 27.34it/s]
accuracy: 0.3070, preci

{'best_epoch': 5,
 'best_validation_accuracy': 0.33787465940054495,
 'best_validation_f1_measure': 0.0,
 'best_validation_loss': 1.5015768187386649,
 'best_validation_precision': 0.0,
 'best_validation_recall': 0.0,
 'epoch': 39,
 'peak_cpu_memory_MB': 589.088,
 'peak_gpu_0_memory_MB': 10,
 'training_accuracy': 0.9321161048689138,
 'training_cpu_memory_MB': 589.088,
 'training_duration': '0:06:45.719439',
 'training_epochs': 39,
 'training_f1_measure': 0.9584934711456299,
 'training_gpu_0_memory_MB': 10,
 'training_loss': 0.20958990990864426,
 'training_precision': 0.9490106701850891,
 'training_recall': 0.9681677222251892,
 'training_start_epoch': 0,
 'validation_accuracy': 0.34514078110808355,
 'validation_f1_measure': 0.32826748490333557,
 'validation_loss': 3.9350390911102293,
 'validation_precision': 0.32926830649375916,
 'validation_recall': 0.3272727131843567}

## Sanity Check

The cell below will allow you to enter sample sentences and test the predictions of the model.

In [24]:
predictor = SentenceClassifierPredictor(model, dataset_reader=reader)
logits = predictor.predict("Don't waste your money")['logits']
label_id = np.argmax(logits)

print(model.vocab.get_token_from_index(label_id, 'labels'))

1


## More Substantive Checks

In order to do some more in depth checks how well the model does, and how well it might generalize we can utilize a set of Amazon reviews. 

http://jmcauley.ucsd.edu/data/amazon/

The site above holds a very large of Amazon reviews that can be used for scientific purposes. 

### Task 1: Choose and Download a Subcategory

From the table below, choose a category that you will use for testing. 
Download the 5 core links that hold the full text, title and rating of a review. 


<html>

<table>
<tbody><tr>
  <td>Books</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_10.json.gz">10-core</a> (4,701,968 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz">5-core</a> (8,898,041 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Books.csv">ratings only</a> (22,507,155 ratings)</td>
</tr>

<tr>
  <td>Electronics</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_10.json.gz">10-core</a> (347,393 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz">5-core</a> (1,689,188 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Electronics.csv">ratings only</a> (7,824,482 ratings)</td>
</tr>

<tr>
  <td>Movies and TV</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_10.json.gz">10-core</a> (958,986 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_5.json.gz">5-core</a> (1,697,533 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Movies_and_TV.csv">ratings only</a> (4,607,047 ratings)</td>
</tr>

<tr>
  <td>CDs and Vinyl</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_CDs_and_Vinyl_10.json.gz">10-core</a> (445,412 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_CDs_and_Vinyl_5.json.gz">5-core</a> (1,097,592 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_CDs_and_Vinyl.csv">ratings only</a> (3,749,004 ratings)</td>
</tr>

<tr>
  <td>Clothing, Shoes and Jewelry</td>
  <!-- <td></td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Clothing_Shoes_and_Jewelry_5.json.gz">5-core</a> (278,677 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Clothing_Shoes_and_Jewelry.csv">ratings only</a> (5,748,920 ratings)</td>
</tr>

<tr>
  <td>Home and Kitchen</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_10.json.gz">10-core</a> (25,445 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_5.json.gz">5-core</a> (551,682 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Home_and_Kitchen.csv">ratings only</a> (4,253,926 ratings)</td>
</tr>

<tr>
  <td>Kindle Store</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_10.json.gz">10-core</a> (367,478 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz">5-core</a> (982,619 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Kindle_Store.csv">ratings only</a> (3,205,467 ratings)</td>
</tr>

<tr>
  <td>Sports and Outdoors</td>
  <!-- <td></td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz">5-core</a> (296,337 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Sports_and_Outdoors.csv">ratings only</a> (3,268,695 ratings)</td>
</tr>

<tr>
  <td>Cell Phones and Accessories</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_10.json.gz">10-core</a> (1,854 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz">5-core</a> (194,439 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Cell_Phones_and_Accessories.csv">ratings only</a> (3,447,249 ratings)</td>
</tr>

<tr>
  <td>Health and Personal Care</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Health_and_Personal_Care_10.json.gz">10-core</a> (55,076 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Health_and_Personal_Care_5.json.gz">5-core</a> (346,355 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Health_and_Personal_Care.csv">ratings only</a> (2,982,326 ratings)</td>
</tr>

<tr>
  <td>Toys and Games</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_10.json.gz">10-core</a> (18,637 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz">5-core</a> (167,597 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Toys_and_Games.csv">ratings only</a> (2,252,771 ratings)</td>
</tr>

<tr>
  <td>Video Games</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_10.json.gz">10-core</a> (52,158 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz">5-core</a> (231,780 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Video_Games.csv">ratings only</a> (1,324,753 ratings)</td>
</tr>

<tr>
  <td>Tools and Home Improvement</td>
  <!-- <td></td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Tools_and_Home_Improvement_5.json.gz">5-core</a> (134,476 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Tools_and_Home_Improvement.csv">ratings only</a> (1,926,047 ratings)</td>
</tr>

<tr>
  <td>Beauty</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_10.json.gz">10-core</a> (28,798 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz">5-core</a> (198,502 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Beauty.csv">ratings only</a> (2,023,070 ratings)</td>
</tr>

<tr>
  <td>Apps for Android</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android_10.json.gz">10-core</a> (264,050 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android_5.json.gz">5-core</a> (752,937 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Apps_for_Android.csv">ratings only</a> (2,638,172 ratings)</td>
</tr>

<tr>
  <td>Office Products</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_10.json.gz">10-core</a> (25,374 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz">5-core</a> (53,258 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Office_Products.csv">ratings only</a> (1,243,186 ratings)</td>
</tr>

<tr>
  <td>Pet Supplies</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_10.json.gz">10-core</a> (3,152 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz">5-core</a> (157,836 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Pet_Supplies.csv">ratings only</a> (1,235,316 ratings)</td>
</tr>

<tr>
  <td>Automotive</td>
  <!-- <td></td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Automotive_5.json.gz">5-core</a> (20,473 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Automotive.csv">ratings only</a> (1,373,768 ratings)</td>
</tr>

<tr>
  <td>Grocery and Gourmet Food</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Grocery_and_Gourmet_Food_10.json.gz">10-core</a> (37,348 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Grocery_and_Gourmet_Food_5.json.gz">5-core</a> (151,254 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Grocery_and_Gourmet_Food.csv">ratings only</a> (1,297,156 ratings)</td>
</tr>

<tr>
  <td>Patio, Lawn and Garden</td>
  <!-- <td></td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Patio_Lawn_and_Garden_5.json.gz">5-core</a> (13,272 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Patio_Lawn_and_Garden.csv">ratings only</a> (993,490 ratings)</td>
</tr>

<tr>
  <td>Baby</td>
  <!-- <td></td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Baby_5.json.gz">5-core</a> (160,792 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Baby.csv">ratings only</a> (915,446 ratings)</td>
</tr>

<tr>
  <td>Digital Music</td>
  <!-- <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_10.json.gz">10-core</a> (22,772 reviews)</td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_5.json.gz">5-core</a> (64,706 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Digital_Music.csv">ratings only</a> (836,006 ratings)</td>
</tr>

<tr>
  <td>Musical Instruments</td>
  <!-- <td></td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz">5-core</a> (10,261 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Musical_Instruments.csv">ratings only</a> (500,176 ratings)</td>
</tr>

<tr>
  <td>Amazon Instant Video</td>
  <!-- <td></td> -->
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz">5-core</a> (37,126 reviews)</td>
  <td><a href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Amazon_Instant_Video.csv">ratings only</a> (583,933 ratings)</td>
</tr>
</tbody>
    </table>
</html>

### Task 2

Sanity checks with the reviews.

The format of the files is as follows:

`{
	"reviewerID": "A2ICI6VUC0U5K6",
	"asin": "B0014JKKGK",
	"reviewerName": "Jermin Botrous \"gigigigi\"",
	"helpful": [0, 0],
	"reviewText": "Don't waste your money because elastic goes bad after 2 washes",
	"overall": 1.0,
	"summary": "One Star",
	"unixReviewTime": 1404432000,
	"reviewTime": "07 4, 2014"
}`

Use the following code snippets to load individal review texts.

Opening a file in python:

``
test_file = open('file_name.json', 'r')
first_line = test_file.readline()
``

Transform the line into a json object to access the individual fiels (such as reviewText).


``
import json
j_obj = json.loads(first_line)
print('reviewText:' + j_obj['reviewText'])
``

finally use the code from above to test the predictions

``
logits = predictor.predict("Don't waste your money")['logits']
label_id = np.argmax(logits)
prediction = model.vocab.get_token_from_index(label_id, 'labels')
print(prediction)
``

In [25]:
test_file = open('../reviews_Digital_Music_5.json', 'r')
first_line = test_file.readline()
print(first_line)

{"reviewerID": "A3EBHHCZO6V2A4", "asin": "5555991584", "reviewerName": "Amaranth \"music fan\"", "helpful": [3, 3], "reviewText": "It's hard to believe \"Memory of Trees\" came out 11 years ago;it has held up well over the passage of time.It's Enya's last great album before the New Age/pop of \"Amarantine\" and \"Day without rain.\" Back in 1995,Enya still had her creative spark,her own voice.I agree with the reviewer who said that this is her saddest album;it is melancholy,bittersweet,from the opening title song.\"Memory of Trees\" is elegaic&majestic.;\"Pax Deorum\" sounds like it is from a Requiem Mass,it is a dark threnody.Unlike the reviewer who said that this has a \"disconcerting\" blend of spirituality&sensuality;,I don't find it disconcerting at all.\"Anywhere is\" is a hopeful song,looking to possibilities.\"Hope has a place\" is about love,but it is up to the listener to decide if it is romantic,platonic,etc.I've always had a soft spot for this song.\"On my way home\" is a t

In [26]:
import json 
j_obj = json.loads(first_line) 
print('reviewText:' + j_obj['reviewText'])

reviewText:It's hard to believe "Memory of Trees" came out 11 years ago;it has held up well over the passage of time.It's Enya's last great album before the New Age/pop of "Amarantine" and "Day without rain." Back in 1995,Enya still had her creative spark,her own voice.I agree with the reviewer who said that this is her saddest album;it is melancholy,bittersweet,from the opening title song."Memory of Trees" is elegaic&majestic.;"Pax Deorum" sounds like it is from a Requiem Mass,it is a dark threnody.Unlike the reviewer who said that this has a "disconcerting" blend of spirituality&sensuality;,I don't find it disconcerting at all."Anywhere is" is a hopeful song,looking to possibilities."Hope has a place" is about love,but it is up to the listener to decide if it is romantic,platonic,etc.I've always had a soft spot for this song."On my way home" is a triumphant ending about return.This is truly a masterpiece of New Age music,a must for any Enya fan!


In [27]:
logits = predictor.predict("Don't waste your money")['logits']
label_id = np.argmax(logits)
prediction = model.vocab.get_token_from_index(label_id, 'labels')
print(prediction)

1


## Task 3

Iterate over the reviews and extract:

* 100 positive predictions (i.e. 4)
* 100 negative predictions (i.e. 0)

Save the sets of positive and negative predictions as plain text files:

* categoryName_100_pos.txt
* categoryName_100_neg.txt

Manually inspect the predictions to identify potential false positives in boths sets.
Store a couple of those false positives in the files:

* categoryName_100_pos_fp.txt
* categoryName_100_neg_fp.txt

In [28]:
test_file = open('../reviews_Digital_Music_5.json', 'r')
lines = test_file.readlines()
positives = []
negatives = []
for line in lines:
  j_obj = json.loads(line) 
  logits = predictor.predict(j_obj['reviewText'])['logits']
  label_id = np.argmax(logits)
  prediction = model.vocab.get_token_from_index(label_id, 'labels')
  print(prediction)
  if prediction == '4' and len(positives) < 100:
    positives.append(line)
    print("Pos len : "+ str(len(positives)))
  elif prediction == '0' and len(negatives) < 100:
    negatives.append(line)
    print("Neg len : "+ str(len(negatives)))
  if len(positives) >= 100 and len(negatives) >= 100:
    break


3
3
0
Neg len : 1
2
1
4
Pos len : 1
0
Neg len : 2
1
3
0
Neg len : 3
3
4
Pos len : 2
0
Neg len : 4
1
4
Pos len : 3
0
Neg len : 5
0
Neg len : 6
4
Pos len : 4
3
4
Pos len : 5
0
Neg len : 7
4
Pos len : 6
4
Pos len : 7
3
3
2
2
3
2
1
0
Neg len : 8
2
2
1
0
Neg len : 9
1
3
4
Pos len : 8
3
3
4
Pos len : 9
2
4
Pos len : 10
2
2
1
3
4
Pos len : 11
2
3
0
Neg len : 10
2
4
Pos len : 12
1
0
Neg len : 11
0
Neg len : 12
3
4
Pos len : 13
0
Neg len : 13
3
0
Neg len : 14
3
2
3
2
3
0
Neg len : 15
3
0
Neg len : 16
3
3
1
2
1
2
3
2
1
1
3
2
3
3
0
Neg len : 17
4
Pos len : 14
3
1
4
Pos len : 15
1
0
Neg len : 18
0
Neg len : 19
1
4
Pos len : 16
3
0
Neg len : 20
1
4
Pos len : 17
0
Neg len : 21
3
0
Neg len : 22
0
Neg len : 23
1
2
0
Neg len : 24
1
1
2
0
Neg len : 25
2
4
Pos len : 18
1
1
1
0
Neg len : 26
2
1
3
3
1
0
Neg len : 27
0
Neg len : 28
2
2
3
1
4
Pos len : 19
3
1
3
4
Pos len : 20
3
3
1
2
3
0
Neg len : 29
1
3
0
Neg len : 30
0
Neg len : 31
2
3
3
0
Neg len : 32
4
Pos len : 21
3
0
Neg len : 33
3
1
2
0
Neg len : 34
4

In [29]:
print(positives)

['{"reviewerID": "A2P49WD75WHAG5", "asin": "5555991584", "reviewerName": "Daniel J. Hamlow", "helpful": [62, 65], "reviewText": "Who knows why I initially considered this to be a duff album by the woman with the most beautiful voice in the world.  Maybe I was expecting Shepherd\'s Moon Part II and saw Memory Of Trees as Shepherd\'s Eclipse.  The songs didn\'t leap out to me?  Well, listening to it for this review, it finally did, and I\'ve embraced it like I have her other albums.  The four years inbetween albums was worth it.Of the opening instrumental title tracks, &quot;The Memory Of Trees&quot; is the most potent, sporting the usual instrumentals, pianos, haunting wall-of-sound choir-like vocals, pounding drums recalling &quot;The Longships&quot; from Watermark.The brisk &quot;Anywhere Is&quot;, whose dominant tempo reminds me of a horse having a gentle canter down a park, is backed by strings and backing vocals.  There\'s an interesting motif in the first line of her reaching a ho

In [0]:
positive_file = open('../categoryName_100_pos.txt', 'w+')
for line in positives:
  positive_file.write(line)

In [0]:
negative_file = open('../categoryName_100_neg.txt', 'w+')
for line in negatives:
  negative_file.write(line)

## Task 4

Generate listing of false positives.
Analyse the data from Amazon.
What would be a way to utilize this data in order to generate larger lists of false positives?
Derive a method that will allow you to predict over the full content of the file and create lists of:
* True positive 'positive' predictions
* False positive 'positive' predictions
* True positive 'negative' predictions
* False positive 'negative' predictions

Save the four sets four your group submission:

* categoryName_pos_tp.txt
* categoryName_pos_fp.txt
* categoryName_neg_tp.txt
* categoryName_neg_fp.txt



In [32]:
test_file = open('../reviews_Digital_Music_5.json', 'r')
lines = test_file.readlines()
pos_tp = []
pos_fp = []
neg_tp = []
neg_fp = []
for line in lines:
  try:
    j_obj = json.loads(line)
    logits = predictor.predict(j_obj['reviewText'])['logits']
    label_id = np.argmax(logits)
    prediction = model.vocab.get_token_from_index(label_id, 'labels')
    if prediction == '4':
      if j_obj['overall'] > 3:
        pos_tp.append(line)
        #print("POS TP : "+ str(len(pos_tp)))
      else:
        pos_fp.append(line)
        #print("POS FP : "+ str(len(pos_fp)))
    elif prediction == '0':
      if j_obj['overall'] <= 3:
        neg_tp.append(line)
        #print("NEG TP : "+ str(len(neg_tp)))
      else:
        neg_fp.append(line)
        #print("NEG FP : "+ str(len(neg_fp)))
  except:
    print(line)
  

NEG FP : 1
POS TP : 1
NEG TP : 1
NEG FP : 2
POS TP : 2
NEG FP : 3
POS TP : 3
NEG FP : 4
NEG FP : 5
POS TP : 4
POS TP : 5
NEG FP : 6
POS TP : 6
POS TP : 7
NEG TP : 2
NEG FP : 7
POS TP : 8
POS TP : 9
POS TP : 10
POS TP : 11
NEG TP : 3
POS TP : 12
NEG FP : 8
NEG FP : 9
POS TP : 13
NEG FP : 10
NEG FP : 11
NEG FP : 12
NEG FP : 13
NEG FP : 14
POS TP : 14
POS TP : 15
NEG FP : 15
NEG FP : 16
POS TP : 16
NEG TP : 4
POS TP : 17
NEG FP : 17
NEG FP : 18
NEG FP : 19
NEG FP : 20
NEG FP : 21
POS TP : 18
NEG FP : 22
NEG FP : 23
NEG FP : 24
POS TP : 19
POS TP : 20
NEG TP : 5
NEG FP : 25
NEG FP : 26
NEG FP : 27
POS TP : 21
NEG FP : 28
NEG FP : 29
POS TP : 22
NEG FP : 30
NEG FP : 31
NEG FP : 32
POS TP : 23
NEG FP : 33
POS TP : 24
POS TP : 25
NEG TP : 6
POS TP : 26
NEG FP : 34
POS TP : 27
POS TP : 28
NEG FP : 35
POS TP : 29
POS TP : 30
NEG TP : 7
NEG FP : 36
POS TP : 31
POS TP : 32
POS TP : 33
NEG FP : 37
POS TP : 34
POS TP : 35
POS TP : 36
NEG FP : 38
NEG FP : 39
POS TP : 37
POS TP : 38
NEG TP : 8
NEG FP

In [0]:
pos_tp_file = open('../categoryName_pos_tp.txt', 'w+')
pos_fp_file = open('../categoryName_pos_fp.txt', 'w+')
neg_tp_file = open('../categoryName_neg_tp.txt', 'w+')
neg_fp_file = open('../categoryName_neg_fp.txt', 'w+')
for line in pos_tp:
  pos_tp_file.write(line)
for line in pos_fp:
  pos_fp_file.write(line)
for line in neg_tp:
  neg_tp_file.write(line)
for line in neg_fp:
  neg_fp_file.write(line)

## Task 5 

Calculate approximate precision values based on your mapping from task 4.

Store the calculations as part of a readme or send the values by e-mail submission. 

Submit the files from Task 3 and Task 4 as your group submission.



In [41]:
#(truepos + trueneg) / total amount = precision

pos_tp_file = open('../categoryName_pos_tp.txt', 'r')
pos_fp_file = open('../categoryName_pos_fp.txt', 'r')
neg_tp_file = open('../categoryName_neg_tp.txt', 'r')
neg_fp_file = open('../categoryName_neg_fp.txt', 'r')

amount_pos_tp = len(pos_tp_file.readlines())
print(amount_pos_tp)

amount_pos_fp = len(pos_fp_file.readlines())
print(amount_pos_fp)

amount_neg_tp = len(neg_tp_file.readlines())
print(amount_neg_tp)

amount_neg_fp = len(neg_fp_file.readlines())
print(amount_neg_fp)

amount_truePredicted = 0
amount_wrongPredicted = 0

amount_truePredicted += amount_pos_tp
amount_truePredicted += amount_neg_tp

amount_wrongPredicted += amount_pos_fp 
amount_wrongPredicted += amount_neg_fp
total = amount_truePredicted + amount_wrongPredicted

precision = amount_truePredicted/total
print(precision)

11221
992
4230
10455
0.5744293256004164
